In [56]:
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
import pandas

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
input_bucket = 's3://hadoop-data-tja'
input_path = '/hadoop/*.json'
#df = spark.read.schema(schema).json(input_bucket + input_path).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
df = spark.read.option("multiline",True).json("s3://hadoop-data-tja/hadoop/hadoop/*.json")
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _corrupt_record: string (nullable = true)
 |-- body: struct (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- account_data: struct (nullable = true)
 |    |    |    |-- contributors_enabled: boolean (nullable = true)
 |    |    |    |-- created_at: string (nullable = true)
 |    |    |    |-- default_profile: boolean (nullable = true)
 |    |    |    |-- default_profile_image: boolean (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- entities: struct (nullable = true)
 |    |    |    |    |-- description: struct (nullable = true)
 |    |    |    |    |    |-- urls: array (nullable = true)
 |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |    |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |    |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    

In [59]:
schema = df.schema
true_schema = schema[1:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
df = spark.read.schema(true_schema).option("multiline",True).json("s3://hadoop-data-tja/hadoop/hadoop/*.json")
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- body: struct (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- account_data: struct (nullable = true)
 |    |    |    |-- contributors_enabled: boolean (nullable = true)
 |    |    |    |-- created_at: string (nullable = true)
 |    |    |    |-- default_profile: boolean (nullable = true)
 |    |    |    |-- default_profile_image: boolean (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- entities: struct (nullable = true)
 |    |    |    |    |-- description: struct (nullable = true)
 |    |    |    |    |    |-- urls: array (nullable = true)
 |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |    |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |    |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |    |    |    |    |-- element: long (contains

In [61]:
df.filter(df.body.isNull()).count(), df.filter(df.body.isNotNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(107, 458)

In [62]:
df2 = df.filter(df.body.isNotNull())
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+
|                body|extractionTimestamp|
+--------------------+-------------------+
|[[[false, Mon Aug...|         1640724670|
|[[[false, Mon Aug...|         1640728269|
|[[[false, Mon Aug...|         1640735013|
|[[[false, Wed Nov...|         1638197526|
|[[[false, Wed Nov...|         1638190328|
|[[[false, Wed Nov...|         1638193926|
|[[[false, Wed Nov...|         1640861341|
|[[[false, Wed Nov...|         1640857743|
|[[[false, Wed Nov...|         1640864942|
|[[[false, Wed Nov...|         1638135191|
|[[[false, Wed Nov...|         1638138792|
|[[[false, Thu Oct...|         1638135180|
|[[[false, Thu Oct...|         1638193915|
|[[[false, Thu Oct...|         1638190317|
|[[[false, Thu Oct...|         1638197515|
|[[[false, Thu Oct...|         1638138781|
|[[[false, Thu Oct...|         1640728270|
|[[[false, Thu Oct...|         1640724671|
|[[[false, Thu Oct...|         1640864931|
|[[[false, Thu Oct...|         1640861330|
+----------

In [10]:
df2.select('body.data.account_data.created_at', 'body.data.account_data.description', 'body.data.account_data.id').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+
|          created_at|         description|        id|
+--------------------+--------------------+----------+
|Mon Aug 11 14:41:...|U.S. Senator from...|  15808765|
|Mon Aug 11 14:41:...|U.S. Senator from...|  15808765|
|Mon Aug 11 14:41:...|U.S. Senator from...|  15808765|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Wed Nov 19 20:10:...|Official Account ...|  17494010|
|Thu Oct 31 19:02:...|The U.S. Senate O...|2167097881|
|Thu Oct 31 19:02:...|The U.S. Senate O...|2167097881|
|Thu Oct 31 19:02:...|The U.S. Senate O...|2167097881|
|Thu Oct 31 19:02:...|The U.S. Senate O...|2167097881|
|Thu Oct 3

# Get only the latest downloaded profiles in cases of redundancies

In [63]:
df3 = df2.withColumn("id", df2['body.data.account_data.id'])
df3 = df2.withColumn("user", df2['body.user'])
df3 = df3.withColumn("name", df3['body.data.account_data.name'])
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+----------+----------------+
|                body|extractionTimestamp|      user|            name|
+--------------------+-------------------+----------+----------------+
|[[[false, Mon Aug...|         1640724670|CoryBooker|     Cory Booker|
|[[[false, Mon Aug...|         1640728269|CoryBooker|     Cory Booker|
|[[[false, Mon Aug...|         1640735013|CoryBooker|     Cory Booker|
|[[[false, Wed Nov...|         1638197526|SenSchumer|   Chuck Schumer|
|[[[false, Wed Nov...|         1638190328|SenSchumer|   Chuck Schumer|
|[[[false, Wed Nov...|         1638193926|SenSchumer|   Chuck Schumer|
|[[[false, Wed Nov...|         1640861341|SenSchumer|   Chuck Schumer|
|[[[false, Wed Nov...|         1640857743|SenSchumer|   Chuck Schumer|
|[[[false, Wed Nov...|         1640864942|SenSchumer|   Chuck Schumer|
|[[[false, Wed Nov...|         1638135191|SenSchumer|   Chuck Schumer|
|[[[false, Wed Nov...|         1638138792|SenSchumer|   Chuck Schumer|
|[[[fa

In [64]:
duplicated = df3.dropDuplicates(["user"])
duplicated.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+---------------+--------------------+
|                body|extractionTimestamp|           user|                name|
+--------------------+-------------------+---------------+--------------------+
|[[[false, Wed Nov...|         1638197526|     SenSchumer|       Chuck Schumer|
|[[[false, Tue Jul...|         1640728277|SenatorTomUdall|Archive: Senator ...|
|[[[false, Tue Jul...|         1640734119|    TomCottonAR|          Tom Cotton|
|[[[false, Tue Nov...|         1640724667|  SenatorHassan|  Sen. Maggie Hassan|
|[[[false, Sat Aug...|         1640870169|      JeffFlake|          Jeff Flake|
|[[[false, Tue Jan...|         1640790420|SenatorMenendez|Senator Bob Menendez|
|[[[false, Mon Jun...|         1638190326|  TomUdallPress|     Tom Udall Press|
|[[[false, Fri Sep...|         1638137891|DianneFeinstein|    Dianne Feinstein|
|[[[false, Thu Aug...|         1638190314|  Maggie_Hassan|       Maggie Hassan|
|[[[false, Mon Aug...|         164072467

In [65]:
duplicated.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

28

Only 28 users were really tweeting and active during the time we've garnered the tweets.

In [66]:
duplicated2 = df3.dropDuplicates(["name"])
duplicated2.show(), duplicated2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+---------------+--------------------+
|                body|extractionTimestamp|           user|                name|
+--------------------+-------------------+---------------+--------------------+
|[[[false, Tue Jul...|         1640728277|SenatorTomUdall|Archive: Senator ...|
|[[[false, Fri Sep...|         1638137891|DianneFeinstein|    Dianne Feinstein|
|[[[false, Thu Dec...|         1638196613|  lisamurkowski| Sen. Lisa Murkowski|
|[[[false, Thu Aug...|         1638190314|  Maggie_Hassan|       Maggie Hassan|
|[[[false, Wed Dec...|         1640734109|   SenDougJones|Archived: Senator...|
|[[[false, Wed Jun...|         1640860429|  LisaForSenate|      Lisa Murkowski|
|[[[false, Wed Feb...|         1640778381|    SenatorBurr|        Richard Burr|
|[[[false, Tue Jan...|         1640790420|SenatorMenendez|Senator Bob Menendez|
|[[[false, Thu Jan...|         1640857737| MartinHeinrich|     Martin Heinrich|
|[[[false, Tue Jul...|         164073411

In [67]:
duplicated.select("name").dropDuplicates().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

25

In [69]:
duplicated.select(['user', 'name']).groupby(['name']).count().where('count > 1').sort('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+
|           name|count|
+---------------+-----+
|     Tom Cotton|    2|
|Martin Heinrich|    2|
|  Chuck Schumer|    2|
+---------------+-----+

3 of those 25 active people have multiple accounts, some for their teams, some private versus the public ones.

In [73]:
duplicated.filter('name = "Tom Cotton" or name = "Martin Heinrich" or name = "Chuck Schumer"').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------------+---------------+
|                body|extractionTimestamp|          user|           name|
+--------------------+-------------------+--------------+---------------+
|[[[false, Wed Nov...|         1638197526|    SenSchumer|  Chuck Schumer|
|[[[false, Tue Jul...|         1640734119|   TomCottonAR|     Tom Cotton|
|[[[false, Sat Nov...|         1640870172|  SenTomCotton|     Tom Cotton|
|[[[false, Mon Sep...|         1638197527|  chuckschumer|  Chuck Schumer|
|[[[false, Thu Aug...|         1640864937|  TeamHeinrich|Martin Heinrich|
|[[[false, Thu Jan...|         1640857737|MartinHeinrich|Martin Heinrich|
+--------------------+-------------------+--------------+---------------+

In [74]:
duplicated = duplicated.withColumn("favourites_count", df2['body.data.account_data.favourites_count'])
duplicated = duplicated.withColumn("followers_count", df2['body.data.account_data.followers_count']) 
duplicated = duplicated.withColumn("friends_count", df2['body.data.account_data.friends_count']) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1641119298972_0001,pyspark,idle,Link,Link,✔


In [54]:
duplicated.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------------------+--------------------+----------------+---------------+-------------+-----------+---------------+
|                body|extractionTimestamp|                id|                name|favourites_count|followers_count|friends_count|geo_enabled|           user|
+--------------------+-------------------+------------------+--------------------+----------------+---------------+-------------+-----------+---------------+
|[[[false, Mon Sep...|         1640790431|          72198806|  Kirsten Gillibrand|             814|        1605545|         1085|       true|  SenGillibrand|
|[[[false, Tue Nov...|         1640724667|         946549322|  Sen. Maggie Hassan|             261|         159425|          542|      false|  SenatorHassan|
|[[[false, Wed Feb...|         1640778381|          21157904|        Richard Burr|              59|         173220|          862|       true|    SenatorBurr|
|[[[false, Sat Nov...|         1640870172|         9

In [33]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version 
-------------------------- --------
beautifulsoup4             4.9.1   
boto                       2.49.0  
click                      7.1.2   
Cython                     0.29.26 
jmespath                   0.10.0  
joblib                     0.15.1  
lxml                       4.5.1   
mysqlclient                1.4.2   
nltk                       3.5     
nose                       1.3.4   
numpy                      1.16.5  
packaging                  21.3    
pandas                     0.25.1  
pip                        9.0.1   
py-dateutil                2.2     
pybind11                   2.9.0   
pyparsing                  3.0.6   
python-dateutil            2.8.2   
python37-sagemaker-pyspark 1.3.0   
pytz                       2020.1  
PyYAML                     5.3.1   
regex                      2020.6.8
setuptools                 28.8.0  
six                        1.13.0  
soupsieve                  1.9.5   
tqdm                       4

In [100]:
duplicated.createOrReplaceTempView("duplicated")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
%%sql -o duplicated -n -1
SELECT user, retweet_count as retweets, favourites_count AS favourites, followers_count AS followers, friends_count AS friends from duplicated ORDER BY favourites_count Desc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [50]:
accounts = spark.read.csv("s3://hadoop-data-tja/*.csv")
accounts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+-----+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|      _c0|     _c1|  _c2|            _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|               _c9|
+---------+--------+-----+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|firstname|lastname|party|       username|         description|          created_at|                name|            location|                 url|                id|
|   Shelby| Richard|    R|      SenShelby|The official Twit...|2009-02-17 17:53:...|      Richard Shelby| Tuscaloosa, Alabama|https://t.co/1iUL...|          21111098|
|    Jomes|    Doug|    D|   SenDougJones|Latest news from ...|2017-12-13 22:59:...|Archived: Senator...|      Birmingham, AL|https://t.co/ybmv...|941080085121175552

In [75]:
duplicated = duplicated.withColumn("geo_enabled", df2['body.data.account_data.geo_enabled']) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
duplicated = duplicated.withColumn("retweet_count", df2['body.data.account_data.status.retweet_count'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
accounts = accounts.withColumnRenamed("_c3","user") 
accounts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+-----+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|      _c0|     _c1|  _c2|           user|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|               _c9|
+---------+--------+-----+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|firstname|lastname|party|       username|         description|          created_at|                name|            location|                 url|                id|
|   Shelby| Richard|    R|      SenShelby|The official Twit...|2009-02-17 17:53:...|      Richard Shelby| Tuscaloosa, Alabama|https://t.co/1iUL...|          21111098|
|    Jomes|    Doug|    D|   SenDougJones|Latest news from ...|2017-12-13 22:59:...|Archived: Senator...|      Birmingham, AL|https://t.co/ybmv...|941080085121175552

In [82]:
duplicated = duplicated.join(accounts,['user'],how='inner')
duplicated = duplicated.withColumnRenamed("_c2","party")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
duplicated.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+-------------------+--------------------+----------------+---------------+-------------+-----------+----------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|           user|                body|extractionTimestamp|                name|favourites_count|followers_count|friends_count|geo_enabled|       _c0|    _c1|party|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|               _c9|
+---------------+--------------------+-------------------+--------------------+----------------+---------------+-------------+-----------+----------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|     SenSchumer|[[[false, Wed Nov...|         1638197526|       Chuck Schumer|             875|        3333807|        24351|      fals

In [85]:
duplicated.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['user', 'body', 'extractionTimestamp', 'name', 'favourites_count', 'followers_count', 'friends_count', 'geo_enabled', '_c0', '_c1', 'party', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9']

In [86]:
duplicated.createOrReplaceTempView("duplicated")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
%%sql -o duplicated -n -1
SELECT party, COUNT(party) as count from duplicated GROUP BY party

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [90]:
%%sql -o duplicated -n -1
SELECT geo_enabled, COUNT(geo_enabled) from duplicated GROUP BY geo_enabled

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [103]:
%%sql -o duplicated -n -1
SELECT party, AVG(retweet_count) as retweets, AVG(followers_count) AS followers, AVG(friends_count) as friends, AVG(favourites_count) AS favourites, COUNT(party) AS count from duplicated GROUP BY party

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [93]:
df2 = df2.withColumn("user", df2['body.user']) 
df2.createOrReplaceTempView("df2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
%%sql -o df2 -n -1
SELECT user, COUNT(user) AS count from df2 GROUP BY user

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()